In [1]:
import numpy as np
import matplotlib.pyplot as plt

from manim import *
green = "#87c2a5"
blue = "#525893"
red = "#e07a5f"
black = "#343434"

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

import ipywidgets as widgets  # interactive display
%config InlineBackend.figure_format = 'retina'
# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/main/nma.mplstyle")
my_layout = widgets.Layout()

<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Phase plane analysis</h1>

##### Example: FitzHugh-Nagumo model
The FitzHugh-Nagumo model is a two-dimensional system of ordinary differential equations that describes the dynamics of a neuron. The defining functions are:
$$\tau\frac{\partial u}{\partial t}=F(u,w) + RI(t) = u-\frac13 u^3-w + RI(t)$$ 
$$\tau\frac{\partial w}{\partial t}=G(u,w) = b_0 + b_1 u - w$$

In [54]:
### Phase plane analysis with pulse inputs

b0 = 2
b1 = 2

class PhasePlane(Scene):
    def construct(self):
        self.camera.background_color = "#ece6e2"

        # Add vector field and nullclines
        self.initiate_phase_plane()

        self.wait(1)
        
        # Add a dot to represent the state of the system
        self.dot = Dot(point=ORIGIN, color=BLUE_D, radius=.1)
        self.play(FadeIn(self.dot))

        # Add axes in the corner to plot the membrane voltage
        self.add_small_axes()

        self.wait(2)
        
        # Move the dot in the vector field
        self.vector_field.nudge(self.dot, 0, 200)
        self.dot.add_updater(self.vector_field.get_nudge_updater())

        def update_plot(mob):
            if self.x.shape[0] >= 100:
                self.x = self.x[1:] - 0.05
                self.y = self.y[1:]

            self.x = np.append(self.x, self.x[-1] + 0.05)
            self.y = np.append(self.y, self.dot.get_center()[0])

            mob.become(
                self.ax_small.plot_line_graph(self.x, self.y, add_vertex_dots = False, line_color = BLUE_D)
            )
            
        self.u_plot.add_updater(update_plot)
        self.wait(7)
        self.clear()

        ### Pulse input
        self.pulse_input(1, update_plot)
        self.clear()

        ### Pulse input
        self.pulse_input(2, update_plot)
        self.clear()

        ### Pulse input
        self.pulse_input(-1, update_plot)
        self.clear()


    def initiate_phase_plane(self):
        #Fitzhugh-Nagumo model
        func = lambda pos: np.array([pos[0] - 1/3*pos[0]**3 - pos[1], b0 + b1*pos[0] - pos[1], 0])

        self.vector_field = ArrowVectorField(
            func, min_color_scheme_value=1, max_color_scheme_value=20, x_range=[-7, 7, 1], y_range=[-4, 4, 1], opacity=0.7
        )

        ax = Axes(
            x_range=[-7, 7, 1],
            y_range=[-4, 4, 1],
            tips=False,
            axis_config = {
                "color": black
                }
        )

        labels = ax.get_axis_labels(
            Tex("u").scale(0.7), Text("w").scale(0.45)
        ).set_color(BLACK)

        self.vector_field.fit_to_coordinate_system(ax)

        # Nullclines
        graph1 = ax.plot(lambda x: x*1.25 - 1/3*x**3, x_range=[-2.85, 2.85], use_smoothing=False, color=RED)
        graph2 = ax.plot(lambda x: b0 + (b1-0.15)*x, x_range=[-3.3, 1.2], use_smoothing=False, color=RED)

        rect = Rectangle(width=4.0, height=2.0, fill_color=RED, color="#ece6e2", fill_opacity=1).set_fill("#ece6e2").shift(UP*3.3+LEFT*4.5)
        title1 = Text('FitzHugh-Nagumo model', color=blue).scale(.5).shift(UP*3.3+LEFT*4.5)
        title2 = Text('Phase plane analysis', color=blue).scale(.5).shift(UP*2.8+LEFT*4.5)

        self.add(self.vector_field, ax, labels, graph1, graph2)
        self.add(rect)
        self.add(title1, title2)

    def add_small_axes(self):
        # Add little axes in the corner
        self.ax_small = Axes(
            x_range = [0, 5, 1],
            y_range = [-2, 2, 1],
            x_length = 8,
            y_length = 5,
            axis_config = {
                "color": black,
                "include_tip": False,
                },
        ).shift(UP*2 + RIGHT*4).scale(0.5)

        labels = self.ax_small.get_axis_labels(
            Tex("t").scale(0.7), Text("u").scale(0.45)
        ).set_color(BLACK)

        self.play(FadeIn(self.ax_small), FadeIn(labels))

        self.x = np.array([0])
        self.y = np.array([0])

        self.u_plot = self.ax_small.plot_line_graph(self.x, self.y, add_vertex_dots = False, line_color = BLUE_D)

        self.add(self.u_plot)
        self.wait(1)

    def clear(self):
        # Clear updaters to start next scene
        self.dot.clear_updaters()
        self.u_plot.clear_updaters()
        self.remove(self.dot)
        self.remove(self.u_plot)

        self.dot = Dot(point=[-1.30,-0.59,0], color=BLUE_D, radius=.1)
        self.add(self.dot)
        
        self.x = np.array([0])
        self.y = np.array([0])
        self.u_plot = self.ax_small.plot_line_graph(self.x, self.y, add_vertex_dots = False, line_color = BLUE_D)
        self.add(self.u_plot)

    def pulse_input(self, a, updater):
        # Pulse input 
        self.play(self.dot.animate.shift(a*RIGHT))

        self.dot.add_updater(self.vector_field.get_nudge_updater())
        self.u_plot.add_updater(updater)
     
        self.wait(7)


# don't remove below command for run button to work
%manim -qm -v ERROR PhasePlane

Manim Community v0.15.1